# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb01089fe80>
├── 'a' --> tensor([[-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222]])
└── 'x' --> <FastTreeValue 0x7fb01089ffd0>
    └── 'c' --> tensor([[ 1.0581,  1.1934,  1.3246, -1.7291],
                        [-1.2374,  1.1112, -0.8893, -0.0939],
                        [ 0.2648,  1.2574,  1.1731, -1.9431]])

In [4]:
t.a

tensor([[-0.0177, -0.1893, -1.1329],
        [-0.8072, -1.8167, -0.0222]])

In [5]:
%timeit t.a

62.9 ns ± 0.0857 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb01089fe80>
├── 'a' --> tensor([[-0.3296, -0.4653,  0.4059],
│                   [ 0.5942, -0.2923,  2.0803]])
└── 'x' --> <FastTreeValue 0x7fb01089ffd0>
    └── 'c' --> tensor([[ 1.0581,  1.1934,  1.3246, -1.7291],
                        [-1.2374,  1.1112, -0.8893, -0.0939],
                        [ 0.2648,  1.2574,  1.1731, -1.9431]])

In [7]:
%timeit t.a = new_value

67.7 ns ± 0.0701 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0177, -0.1893, -1.1329],
               [-0.8072, -1.8167, -0.0222]]),
    x: Batch(
           c: tensor([[ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431]]),
       ),
)

In [10]:
b.a

tensor([[-0.0177, -0.1893, -1.1329],
        [-0.8072, -1.8167, -0.0222]])

In [11]:
%timeit b.a

52.7 ns ± 0.117 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5326, -1.5077,  1.5664],
               [ 0.5682, -1.1460, -1.2123]]),
    x: Batch(
           c: tensor([[ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.182 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

875 ns ± 0.977 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 13.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7faf665bf670>
├── 'a' --> tensor([[[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]],
│           
│                   [[-0.0177, -0.1893, -1.1329],
│                    [-0.8072, -1.8167, -0.0222]]])
└── 'x' --> <FastTreeValue 0x7faf665bf8b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 57.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7faf66562f40>
├── 'a' --> tensor([[-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222],
│                   [-0.0177, -0.1893, -1.1329],
│                   [-0.8072, -1.8167, -0.0222]])
└── 'x' --> <FastTreeValue 0x7fb0108f7dc0>
    └── 'c' --> tensor([[ 1.0581,  1.1934,  1.3246, -1.7291],
                        [-1.2374,  1.1112, -0.8893, -0.0939],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 62.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0581,  1.1934,  1.3246, -1.7291],
                       [-1.2374,  1.1112, -0.8893, -0.0939],
                       [ 0.2648,  1.2574,  1.1731, -1.9431]],
              
                      [[ 1.0581,  1.1934,  1.3246, -1.7291],
                       [-1.2374,  1.1112, -0.8893, -0.0939],
                       [ 0.2648,  1.2574,  1.1731, -1.9431]],
              
                      [[ 1.0581,  1.1934,  1.3246, -1.7291],
                       [-1.2374,  1.1112, -0.8893, -0.0939],
                       [ 0.2648,  1.2574,  1.1731, -1.9431]],
              
                      [[ 1.0581,  1.1934,  1.3246, -1.7291],
                       [-1.2374,  1.1112, -0.8893, -0.0939],
                       [ 0.2648,  1.2574,  1.1731, -1.9431]],
              
                      [[ 1.0581,  1.1934,  1.3246, -1.7291],
                       [-1.2374,  1.1112, -0.8893, -0.0939],
                       [ 0.2648,  1.2574,  1.1731, -1.9431]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 95 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431],
                      [ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431],
                      [ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431],
                      [ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431],
                      [ 1.0581,  1.1934,  1.3246, -1.7291],
                      [-1.2374,  1.1112, -0.8893, -0.0939],
                      [ 0.2648,  1.2574,  1.1731, -1.9431],
                      [ 1.0581,  1.1934,  1.3246, -1.7291],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 343 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
